<a href="https://colab.research.google.com/github/malinphy/os_llms_colab/blob/main/small_llm_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install git+https://github.com/huggingface/transformers
!pip install --upgrade requests torch einops accelerate bitsandbytes
!pip install faiss-cpu -q
!pip install langchain -q
!pip install sentence-transformers -q

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-58wyvoxq
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-58wyvoxq
  Resolved https://github.com/huggingface/transformers to commit 35eac0df75c692c5b93c12f7eaf3279cab8bd7ce
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
import os
os.listdir('drive/MyDrive/QQ_PROJECTS/turkish_pro_OS/data/data/vector_stores')

['sentence_transformers_clip_ViT_B_32_multilingual_v1',
 'cohere_multilingual',
 'faiss_index_OpenAI_pdfs_MC_500_30',
 'emrecan_bert_base_turkish_cased_mean_nli_stsb_tr',
 'muse_3',
 'faiss_index_MCdocs_en_500_30_translated_nllb-200-distilled-600M_embedded_all-MiniLM-L6-v2',
 'translated_faiss_index_helsinki']

In [4]:
from qa import questions
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
import pandas as pd
import numpy as np
from langchain.prompts import PromptTemplate

In [5]:
from transformers import AutoTokenizer, MarianMTModel,AutoModelForSeq2SeqLM
from transformers import MarianMTModel, MarianTokenizer
# from nltk.tokenize import sent_tokenize
tokenizer_eng2tr = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-tr")
model_eng2tr = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-tc-big-en-tr")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [6]:
def tr2eng(input_text):
    src = "tr"  # source language
    trg = "en"  # target language
    model_name_tr2eng = f"Helsinki-NLP/opus-mt-{src}-{trg}"
    model_tr2eng = MarianMTModel.from_pretrained(model_name_tr2eng)
    tokenizer_tr2eng = AutoTokenizer.from_pretrained(model_name_tr2eng)
    batch = tokenizer_tr2eng([input_text], return_tensors="pt")
    generated_ids = model_tr2eng.generate(**batch)
    eng_text = tokenizer_tr2eng.batch_decode(generated_ids, skip_special_tokens=True)[0]
    # print(eng_text)
    return eng_text

In [7]:
def eng2tr(english_text):
    model_name = "Helsinki-NLP/opus-mt-tc-big-en-tr"
    tokenizer_eng2tr = MarianTokenizer.from_pretrained(model_name)
    model_eng2tr = MarianMTModel.from_pretrained(model_name)
    translated = model_eng2tr.generate(**tokenizer_eng2tr(english_text, return_tensors="pt", padding=True))
    decoded = []
    for t in translated:
        decoded.append(tokenizer_eng2tr.decode(t, skip_special_tokens=True) )

    return decoded

In [8]:
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
db = FAISS.load_local('drive/MyDrive/QQ_PROJECTS/turkish_pro_OS/data/data/vector_stores/translated_faiss_index_helsinki',
                      embeddings = embeddings)

In [9]:
# tr2eng(questions['q1'])

In [10]:
# english_questions = []
# for i in questions:
#     # print(i)
#     english_questions.append(tr2eng(questions[i]))

In [11]:
# questions['q2']

In [12]:
# for num, i in enumerate(english_questions):
#     print(num,i)

In [13]:
# english_questions[11]

In [14]:
# similarity_results = db.similarity_search(english_questions[11], k = 10)

In [15]:
# # page_contents = []
# for i in similarity_results:
#     page_contents.append(i.page_content)

In [16]:
# page_contents

## OS results

In [17]:
# db_os = FAISS.load_local('drive/MyDrive/QQ_PROJECTS/turkish_pro_OS/data/data/vector_stores/faiss_index_MCdocs_en_500_30_translated_nllb-200-distilled-600M_embedded_all-MiniLM-L6-v2',
#                       embeddings = embeddings)

# page_contents_os = db_os.similarity_search('What is Nightwork?', k=10)
# page_contents_os_total = []
# for i in page_contents_os:
#     page_contents_os_total.append(i.page_content)

# page_contents_os_total

LLM

In [46]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
import torch

checkpoint = "MBZUAI/LaMini-Flan-T5-783M"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
base_model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint,
                                             device_map='auto',
                                             torch_dtype=torch.float32)

pipe = pipeline('text2text-generation',
                 model = base_model,
                 tokenizer = tokenizer,
                 max_length = 1024,
                 do_sample=True,
                 temperature=0.9,
                 top_p=0.95,
                 )

In [47]:
# multiple_input_prompt = PromptTemplate(
#     input_variables=["Inputs", "Question"],
#     template="{Inputs} {Question}."
# )
# multiple_input_prompt.format(Inputs=" ".join(page_contents[1:]), Question=english_questions[11])

In [48]:
# english_questions[0]

In [49]:
# " ".join(page_contents)

In [50]:
%%time
import textwrap
response = ''
instruction = '''
                 Nightlong and nightwork Article 69- In working life, "night" is a period of eleven hours at the earliest, starting at 8:00 p.m.
                 at the latest. Night work: Article 6 (Default: 17/4/1984-84/7950 K.) The study of a mail that meets more than half of its working
                 period is considered a night's work. Nightlong and nightwork Article 69 is a period in the working life where "night" starts
                 at 8:00 p.m. at the latest, reaching 6:00 p.m. at the earliest, which, in any case, lasts at least 11 hours. If more than half
                 of the study corresponds to the night period, it will not only be the study that coincides with the night period, but the whole
                 work will be considered a night work, it will be subject to the rules of the night work. Article 69 of Job Law 4857 stated that
                 work between 20:00 and 6 p.m. will be considered a night's work. It is important whether a particular part of the study is
                 considered a day or a night's work if a specific part of the night is the same as a period of the day's work. Although tourism,
                 private security and health care may be predicted to work nights with the written approval of workers over seven and a half hours,
                 the work of those workers outside of these workers is estimated to be able to work nights, according to the third joke of the
                 same substance, seven and a half hours a day. According to this, studies on the 7.5 hours prior to 04.04.2015 will be considered
                 a lot of work, and in the night after 4:04.2015 a.m., if the employee's approval record and the working period is not past 45
                 hours per week and the night's work has been completed, the night shift will not be paid to the employees in this sector.

                what are Nightlong and nightwork?
                 '''

# instruction = multiple_input_prompt.format(Inputs=" ".join(page_contents), Question=english_questions[0])
generated_text = pipe(instruction)
for text in generated_text:
  response += text['generated_text']
wrapped_text = textwrap.fill(response, width=1200)
print(wrapped_text)



Yes.
CPU times: user 9.5 s, sys: 10.4 ms, total: 9.51 s
Wall time: 10 s


In [35]:
os_res = " ".join(['WHAT IS THE NIGHT WORKING?',
 "THE WORKER'S WORKER'S WORKER'S WORK IS THE WORK OF A MIDDLE WORKER, MORE THAN half OF THE NIGHT'S WORKER'S TIME OF THE NIGHT'S WORKER'S WORK IS THE WORK OF NIGHT'S WORK.",
 'Nightlong and nightwork Article 69- In working life, "night" is a period of eleven hours at the earliest, starting at 8:00 p.m. at the latest.',
 "Night work: Article 6 (Default: 17/4/1984-84/7950 K.) The study of a mail that meets more than half of its working period is considered a night's work.",
 'Nightlong and nightwork Article 69 is a period in the working life where "night" starts at 8:00 p.m. at the latest, reaching 6:00 p.m. at the earliest, which, in any case, lasts at least 11 hours.',
 'If more than half of the study corresponds to the night period, it will not only be the study that coincides with the night period, but the whole work will be considered a night work, it will be subject to the rules of the night work.',
 "Article 69 of Job Law 4857 stated that work between 20:00 and 6 p.m. will be considered a night's work.",
 "It is important whether a particular part of the study is considered a day or a night's work if a specific part of the night is the same as a period of the day's work.",
 'Although tourism, private security and health care may be predicted to work nights with the written approval of workers over seven and a half hours, the work of those workers outside of these workers is estimated to be able to work nights, according to the third joke of the same substance, seven and a half hours a day.',
 "According to this, studies on the 7.5 hours prior to 04.04.2015 will be considered a lot of work, and in the night after 4:04.2015 a.m., if the employee's approval record and the working period is not past 45 hours per week and the night's work has been completed, the night shift will not be paid to the employees in this sector."
 ])

In [36]:
os_res

'WHAT IS THE NIGHT WORKING? THE WORKER\'S WORKER\'S WORKER\'S WORK IS THE WORK OF A MIDDLE WORKER, MORE THAN half OF THE NIGHT\'S WORKER\'S TIME OF THE NIGHT\'S WORKER\'S WORK IS THE WORK OF NIGHT\'S WORK. Nightlong and nightwork Article 69- In working life, "night" is a period of eleven hours at the earliest, starting at 8:00 p.m. at the latest. Night work: Article 6 (Default: 17/4/1984-84/7950 K.) The study of a mail that meets more than half of its working period is considered a night\'s work. Nightlong and nightwork Article 69 is a period in the working life where "night" starts at 8:00 p.m. at the latest, reaching 6:00 p.m. at the earliest, which, in any case, lasts at least 11 hours. If more than half of the study corresponds to the night period, it will not only be the study that coincides with the night period, but the whole work will be considered a night work, it will be subject to the rules of the night work. Article 69 of Job Law 4857 stated that work between 20:00 and 6 p.